# Custom cumulative 

## While pandas offers range of options, a missing thing is a way to apply any cumulative operation over a serie/a dataframe, let's fix this



In [1]:
import numpy as np
import numba as nb
import pandas as pd
import logging
from typing import Callable, Type, Dict, Tuple, Any

In [2]:
arr = np.random.randn(100_000,6).astype(np.float32)
df = pd.DataFrame(arr, columns = ['A', 'B', 'C', 'D', 'E', 'F'])

##  With axis = 0, cumulative directly apply to each column independantly

We need to set a format, which is new value, previous value + option of a default start value different from 0

In [ ]:


@nb.njit(nb.float32(nb.float32[:]), cache=True, fastmath=False, forceinline=True, looplift=True, inline='always', target_backend='host', nogil=True)
def cumulative_function(new, previous):
    return new + previous if new < previous else new - previous

@nb.njit((nb.float32[:, :], nb.float32[:, :],nb.types.uint32,nb.types.uint32), cache=True, fastmath=True, forceinline=True, looplift=True, inline='always', target_backend='host', nogil=True)
def vl(z, r, i, m):
    for j in nb.prange(m):
        r[i, j] = agg_sumba(z[j, :])


@nb.njit((nb.types.Array(nb.types.float32, 3, 'A', readonly=True), nb.float32[:, :],nb.types.uint32, nb.types.uint32), cache=True, parallel=True, fastmath=True, forceinline=True, looplift=True, inline='always', target_backend='host', no_cfunc_wrapper=True, no_rewrites=True, nogil=True)
def cvlopt(z, r, n, m):
    for i in nb.prange(n):
        vl(z[i,:,:], r, i, m)

def vlopt(arr, window):
    cvlopt(sliding_window_view(arr, window, axis=0), r:=np.empty((n:=arr.shape[0]- window + 1, m:=arr.shape[1]), dtype=np.float32), np.uint32(n), np.uint32(m))
    return r

display(pd.DataFrame(vlopt(df.to_numpy(), window)))
display(df.rolling(window).sum())